In [16]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions
import time
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re
import textract
import os
from unicodedata import normalize
import pandas as pd

options = ChromeOptions()
options.add_argument("headless")
options.add_argument("disable-gpu")

In [26]:
#Scraping data from index report()
marketCaps = []
dates = []
r = requests.get("http://www.crsp.org/fact-sheet-archive")
soup = BeautifulSoup(r.text,features="html.parser")
for link in soup.find_all('a'):
    if re.search('crsptm', str(link.get('href')), re.IGNORECASE):
        pdf = requests.get(link.get('href'))
        with open('/tmp/index.pdf', 'wb') as f:
            f.write(pdf.content)
            read_pdf = textract.process('/tmp/index.pdf')
            read_pdf = read_pdf.decode("utf8")
        os.remove('/tmp/index.pdf')
        pdf_text = re.sub('  ',' ',re.sub('\n',' ',normalize('NFKD',read_pdf)))
        dates.append(link.text.replace('Quarter Ending ', ''))
        marketCaps.append(float(re.findall('INDEX MARKET CAP\s+\d+,\d+,\d+', pdf_text, re.IGNORECASE)[0].upper().replace('INDEX MARKET CAP','').replace(',',''))/1000)
vtiDF = pd.DataFrame({'date':pd.to_datetime(dates, infer_datetime_format=True),'Market Cap':marketCaps}).sort_values(by='date').reset_index(drop=True)
print(vtiDF)

IndexError: list index out of range

In [ ]:
#Scraping data from index report(FTSE Global All Cap ex US Index) for VXUS

In [17]:
#Scraping data from index report(FTSE Russell USBIG Index) for BND
dates = []
marketCaps = []
links = []

driver = webdriver.Chrome(options=options)
driver.get('https://www.ftserussell.com/analytics/factsheets/home/search')
archive = driver.find_element_by_xpath('//a[@title="FTSE US Broad Investment-Grade Bond Index (USBIG®)"]/following-sibling::a')
driver.execute_script("arguments[0].click();",archive)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()
div = soup.find("h4", string="FTSE US Broad Investment-Grade Bond Index (USBIG®)").parent
for li in div.find('ul').findAll('li'):
    links.append(li.find('a',href=True)['href'])
links.append(BeautifulSoup(requests.get('https://www.ftserussell.com/analytics/factsheets/home/search').text,features="html.parser").find('a',{'title':'FTSE US Broad Investment-Grade Bond Index (USBIG®)'})['href'])

for link in links:
    pdf = requests.get(link)
    with open('/tmp/index.pdf', 'wb') as f:
        f.write(pdf.content)
        read_pdf = textract.process('/tmp/index.pdf')
        read_pdf = read_pdf.decode("utf8")
    os.remove('/tmp/index.pdf')
    pdf_text = re.sub('  ',' ',re.sub('\n',' ',normalize('NFKD',read_pdf)))
    dates.append(re.sub('FTSE Russell Factsheet\s\S\s','',re.findall('FTSE Russell Factsheet\s\S\s\w+ \d+, \d+', pdf_text, re.IGNORECASE)[0]))
    marketCaps.append(float(re.sub('Market Value. ','',re.findall('Market Value. \d+,\d+.\d+',pdf_text,re.IGNORECASE)[0]).replace(',','')))
bndDF = pd.DataFrame({'date':pd.to_datetime(dates, infer_datetime_format=True),'Market Cap':marketCaps}).sort_values(by='date').reset_index(drop=True)
print(bndDF)

In [ ]:
#Scraping data from index report(FTSE Russell WorldBIG Index) for BNDX